In [ ]:
import numpy as np
import pandas as pd
import random as rand
import matplotlib.pyplot as plt
import random
import tslearn

In [ ]:
tpath = r'C:\Users\virag\Documents\GitHub\Predictive-Maintenance\cool_tests'
path = r'C:\Users\virag\Documents\GitHub\Predictive-Maintenance\Final_cooling_sys_data'

In [ ]:
dsets = []
for i in range(39):
    dsets.append(pd.read_csv(path + '\\Final_cooling_sys_data/final_cooling_dset_' + str(i)+'.csv'))
for i in range(8):
    dsets.append(pd.read_csv(path + '\\Final_cooling_sys_data/faulty_cooling_dset_' + str(i)+'.csv'))

New